## Feast Client with RBAC

## Feast Kubernetes RBAC Authorization

Feast **Role-Based Access Control (RBAC)** in Kubernetes relies on a **service account** for authentication. This applies both **within a Kubernetes pod** and for **external clients** accessing Feast

In this example, Feast will automatically retrieve the Kubernetes ServiceAccount token from pod path:
```
/var/run/secrets/kubernetes.io/serviceaccount/token
```
This means:
- No manual configuration is needed inside a pod.
- The token is mounted automatically and used for authentication.
- Developer?User just need create the binding with role and service account accordingly.

For more details, refer to the user guide: [Kubernetes RBAC Authorization](https://docs.feast.dev/master/getting-started/components/authz_manager#kubernetes-rbac-authorization). 


###  Feature Store settings
**The Operator create client ConfigMap** containing the `feature_store.yaml `settings. We can retrieve it save it feature_repo folder.

In [14]:
!kubectl get configmap feast-sample-kubernetes-auth-client -n feast -o jsonpath='{.data.feature_store\.yaml}' > client/feature_repo/feature_store.yaml
!cat  client/feature_repo/feature_store.yaml

project: feast_rbac
provider: local
offline_store:
    host: feast-sample-kubernetes-auth-offline.feast.svc.cluster.local
    type: remote
    port: 443
    scheme: https
    cert: /tls/offline/tls.crt
online_store:
    path: https://feast-sample-kubernetes-auth-online.feast.svc.cluster.local:443
    type: remote
    cert: /tls/online/tls.crt
registry:
    path: feast-sample-kubernetes-auth-registry.feast.svc.cluster.local:443
    registry_type: remote
    cert: /tls/registry/tls.crt
auth:
    type: kubernetes
entity_key_serialization_version: 3


### Change the Cert path for all servers to CACert in repo config

Note: Below example is for MacOS, For linux remove empty `''`.

In [15]:
!sed -i '' 's|cert: /tls/[^/]*/tls.crt|cert: /etc/pki/tls/custom-certs/service-ca.crt|g' client/feature_repo/feature_store.yaml

**Create ConfigMap From Feature Repository**  
We need feature_repo inside the container. let's create configmap from `feature_repo` contains the feature repository files, including `feature-store.yaml` and `test.py`. It will be mounted as a volume in the deployment for the client examples to test the script.

In [16]:
!kubectl delete configmap client-feature-repo-config --ignore-not-found -n feast
!kubectl create configmap client-feature-repo-config --from-file=client/feature_repo -n feast

configmap/client-feature-repo-config created


## Testing

### 0. Lets Run Materialization on the Feature Server

In [17]:
!kubectl exec deploy/feast-sample-kubernetes-auth -itc online -- feast materialize -v driver_hourly_stats 2025-05-08T12:00:00Z 2025-05-20T12:00:00Z

/opt/app-root/lib64/python3.11/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "vector_enabled" in "SqliteOnlineStoreConfig" shadows an attribute in parent "VectorStoreConfig"
  warnings.warn(
Materializing 1 feature views from 2025-05-08 12:00:00+00:00 to 2025-05-20 12:00:00+00:00 into the sqlite online store.

driver_hourly_stats:
100%|████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 770.93it/s]


### 1. Test Read-Only Feast User 

**Step 1: Deploy read-only user, we are using `serviceAccountName  feast-user-sa` in deployment.**


In [18]:
# Create the deployment 
!cat client/readonly_user_deployment_tls.yaml
!kubectl apply -f "client/readonly_user_deployment_tls.yaml"

apiVersion: apps/v1
kind: Deployment
metadata:
  name: client-readonly-user
  namespace: feast
  labels:
    app: client-user
spec:
  replicas: 1
  selector:
    matchLabels:
      app: client-user
  template:
    metadata:
      labels:
        app: client-user
    spec:
      serviceAccountName: feast-user-sa
      containers:
        - name: client-user-container
          image: quay.io/feastdev/feature-server:latest
          imagePullPolicy: Always
          command: ["sleep", "infinity"]
          volumeMounts:
            - name: client-feature-repo-config
              mountPath: /opt/app-root/src
            - name: feast-service-ca
              mountPath: /etc/pki/tls/custom-certs/service-ca.crt
              subPath: service-ca.crt
      volumes:
        - name: client-feature-repo-config
          configMap:
            name: client-feature-repo-config
        - name: feast-service-ca
          configMap:
            name: feast-sample-kubernetes-auth-client-cadeployment.

**Step 2: Run test.py script for client-readonly-user, readonly-user can only read or query all objects.**

In [19]:
#Run test.py script from pod to test RBAC for client-readonly-user.
# verify the logs for write operation will show below message 
# --- Write to Feature Store ---
#*** PERMISSION DENIED *** User lacks permission to modify the feature store.

!kubectl exec -n feast -it $(kubectl get pods -n feast -l app=client-user -o jsonpath="{.items[0].metadata.name}") -- python test.py



--- List feature views ---
Successfully listed 2 feature views:
  - driver_hourly_stats_fresh
  - driver_hourly_stats

--- Fetching Historical Features for Training ---
Successfully fetched training historical features:
    driver_id  ... conv_rate_plus_val2
0       1001  ...           10.201450
1       1002  ...           20.377766
2       1003  ...           30.407708

[3 rows x 10 columns]

--- Fetching Historical Features for Batch Scoring ---
Successfully fetched batch scoring historical features:
    driver_id  ... conv_rate_plus_val2
0       1001  ...           10.529917
1       1002  ...           20.251087
2       1003  ...           30.462439

[3 rows x 10 columns]

--- Write to Feature Store ---

*** PERMISSION DENIED *** User lacks permission to modify the feature store.

--- Fetching Online Features ---
Successfully fetched online features directly:

acc_rate : [0.2032364010810852, 0.47476285696029663]
conv_rate_plus_val1 : [1000.6724982261658, 1001.9941676259041]
conv_ra

**Step 3: Run API request for client-readonly-user, readonly-user can only read or query all objects.**

Required:
 - Clients CA Cert from `/etc/pki/tls/custom-certs/service-ca.crt`
 - Users Bearer Token
    - Get the User token from the location `/var/run/secrets/kubernetes.io/serviceaccount/token` OR  `kubectl whoami -t` on the client pod.
    - Replace the <client_user_token> below with token obtained from above.

In [ ]:
# Run Curl command to test the RBAC for client-readonly-user.
!kubectl exec -it $(kubectl get pods -n feast -l app=client-user -o jsonpath="{.items[0].metadata.name}") -n feast -- curl --cacert /etc/pki/tls/custom-certs/service-ca.crt -X POST https://feast-sample-kubernetes-auth-online.feast.svc.cluster.local/get-online-features -H "Content-Type: application/json" -H "Authorization: Bearer <client_user_token>" -d '{"features": ["driver_hourly_stats:conv_rate","driver_hourly_stats:acc_rate"], "entities":{"driver_id": [1001, 1002]}}'

{"metadata":{"feature_names":["driver_id","conv_rate","acc_rate"]},"results":[{"values":[1001,1002],"statuses":["PRESENT","PRESENT"],"event_timestamps":["1970-01-01T00:00:00Z","1970-01-01T00:00:00Z"]},{"values":[0.5884456634521484,0.046199530363082886],"statuses":["PRESENT","PRESENT"],"event_timestamps":["2025-05-20T11:00:00Z","2025-05-20T11:00:00Z"]},{"values":[0.8458412289619446,0.7902957201004028],"statuses":["PRESENT","PRESENT"],"event_timestamps":["2025-05-20T11:00:00Z","2025-05-20T11:00:00Z"]}]}

### 2. Test Unauthorized Feast User 

**Step 1: Run test.py script for client-unauthorized-user, unauthorized-user could not even view all objects.**

In [20]:
!kubectl apply -f "client/unauthorized_user_deployment_tls.yaml"

deployment.apps/client-unauthorized-user created


In [21]:
!kubectl exec -n feast -it $(kubectl get pods -n feast -l app=client-unauthorized-user -o jsonpath="{.items[0].metadata.name}") -- python test.py

/opt/app-root/lib64/python3.11/site-packages/feast/feature_view.py:48: DeprecationWarning: Entity value_type will be mandatory in the next release. Please specify a value_type for entity '__dummy'.
  DUMMY_ENTITY = Entity(

--- List feature views ---
No feature views found. You might not have access or they haven't been created.

--- Fetching Historical Features for Training ---

*** PERMISSION DENIED *** Cannot fetch historical features.

--- Fetching Historical Features for Batch Scoring ---

*** PERMISSION DENIED *** Cannot fetch historical features.

--- Write to Feature Store ---

*** PERMISSION DENIED *** User lacks permission to modify the feature store.

--- Fetching Online Features ---

*** PERMISSION DENIED *** Cannot fetch online features.

--- Fetching Online Features via Feature Service ---

*** PERMISSION DENIED *** Cannot fetch online features.

--- Fetching Online Features via Push Source ---

*** PERMISSION DENIED *** Cannot fetch online features.

--- Performing Push 

**Step 2: Run API request for Unauthorized User, Unauthorized user should not be able to even view the objects.**

Required:
 - Users Bearer Token
    - Get the User token from the location `/var/run/secrets/kubernetes.io/serviceaccount/token` OR  `kubectl whoami -t` on the client pod.
    - Replace the <client_user_token> below with token obtained from above.

In [ ]:
# Run Curl command to test the RBAC for client-readonly-user.
!kubectl exec -it $(kubectl get pods -n feast -l app=client-unauthorized-user -o jsonpath="{.items[0].metadata.name}") -n feast -- curl --cacert /etc/pki/tls/custom-certs/service-ca.crt -X POST https://feast-sample-kubernetes-auth-online.feast.svc.cluster.local/get-online-features -H "Content-Type: application/json" -H "Authorization: Bearer <client_user_token>" -d '{"features": ["driver_hourly_stats:conv_rate","driver_hourly_stats:acc_rate"], "entities":{"driver_id": [1001, 1002]}}'

"{\"module\": \"feast.errors\", \"class\": \"FeastPermissionError\", \"message\": \"Permission error:\\nPermission feast_user_permission denied execution of ['READ_ONLINE'] to FeatureView:driver_hourly_stats: Requires roles ['feast-reader'],Permission feast_admin_permission denied execution of ['READ_ONLINE'] to FeatureView:driver_hourly_stats: Requires roles ['feast-writer']\"}"

## 3. Test Admin Feast User

**Step 1: Run test.py script for clientadmin, client-admin should be perform all operations on all objects.**

In [23]:
!kubectl apply -f "client/admin_user_deployment_tls.yaml"

deployment.apps/client-admin-user created


In [24]:
!kubectl exec -n feast -it $(kubectl get pods -n feast -l app=client-admin -o jsonpath="{.items[0].metadata.name}") -- python test.py



--- List feature views ---
Successfully listed 2 feature views:
  - driver_hourly_stats_fresh
  - driver_hourly_stats

--- Fetching Historical Features for Training ---
Successfully fetched training historical features:
    driver_id  ... conv_rate_plus_val2
0       1001  ...           10.201450
1       1002  ...           20.377766
2       1003  ...           30.407708

[3 rows x 10 columns]

--- Fetching Historical Features for Batch Scoring ---
Successfully fetched batch scoring historical features:
    driver_id  ... conv_rate_plus_val2
0       1001  ...           10.529917
1       1002  ...           20.251087
2       1003  ...           30.462439

[3 rows x 10 columns]

--- Write to Feature Store ---
User has write access to the feature store.

--- Fetching Online Features ---
Successfully fetched online features directly:

acc_rate : [0.2032364010810852, 0.47476285696029663]
conv_rate_plus_val1 : [1000.6724982261658, 1001.9941676259041]
conv_rate_plus_val2 : [2000.6724982261658

**Step 2: Run API request for admin-user, admin-user should be able to read features.**

Required:
 - Clients CA Cert from `/etc/pki/tls/custom-certs/service-ca.crt`
 - Users Bearer Token
    - Get the User token from the location `/var/run/secrets/kubernetes.io/serviceaccount/token` OR  `kubectl whoami -t` on the client pod.
    - Replace the <client_user_token> below with token obtained from above.

In [34]:
# Run Curl command to test the RBAC for client-readonly-user.
!kubectl exec -it $(kubectl get pods -n feast -l app=client-admin -o jsonpath="{.items[0].metadata.name}") -n feast -- curl --cacert /etc/pki/tls/custom-certs/service-ca.crt -X POST https://feast-sample-kubernetes-auth-online.feast.svc.cluster.local/get-online-features -H "Content-Type: application/json" -H "Authorization: Bearer <client_user_token>" -d '{"features": ["driver_hourly_stats:conv_rate","driver_hourly_stats:acc_rate", "transformed_conv_rate:conv_rate_plus_val1"], "entities":{"driver_id": [1001, 1002], "val_to_add": [1000, 1001], "val_to_add_2": [2000, 2002]}}'

{"metadata":{"feature_names":["driver_id","conv_rate","acc_rate","conv_rate_plus_val1"]},"results":[{"values":[1001,1002],"statuses":["PRESENT","PRESENT"],"event_timestamps":["1970-01-01T00:00:00Z","1970-01-01T00:00:00Z"]},{"values":[0.5884456634521484,0.046199530363082886],"statuses":["PRESENT","PRESENT"],"event_timestamps":["2025-05-20T11:00:00Z","2025-05-20T11:00:00Z"]},{"values":[0.8458412289619446,0.7902957201004028],"statuses":["PRESENT","PRESENT"],"event_timestamps":["2025-05-20T11:00:00Z","2025-05-20T11:00:00Z"]},{"values":[1000.5884456634521,1001.0461995303631],"statuses":["PRESENT","PRESENT"],"event_timestamps":["1970-01-01T00:00:00Z","1970-01-01T00:00:00Z"]}]}

[Next: Uninstall](./3-uninstall.ipynb)